In [24]:
import add_packages
import typing, operator, json, os
from toolkit.langchain import (
	chat_models, prompts, graphs, agent_tools, agents
)

from pprint import pprint

ACTION_END = "end"
ACTION_CONTINUE = "continue"
ACTION_FUNCTION_CALL = "function_call"

# [Github](https://github.com/langchain-ai/langgraph/tree/main)

# Class

# Overview

LangGraph: Library for stateful, multi-actor apps with LLMs, built on LangChain. Extends LangChain Expression Language to coordinate multiple chains/actors across multiple steps of computation cyclically. 

The main use of LangGraph is to add cycles to LLM application.

Cycles are crucial for agent-like behaviors, calling an LLM in a loop to determine the next action.

When to Use

- If cycles are needed.
- Langchain Expression Language defines chains easily but lacks a mechanism for cycles. 

# Quick start

LangGraph revolves around the concept of state. During graph execution, a state is passed between nodes and updated by each node with its return value. The method of updating the state is determined by the graph type or a custom function.

State for example is a list of chat messages (LangChain chat models's output) using the MessageGraph class.


In [ ]:
# The graph has a single node named "oracle" that runs a chat model and gives
# back the result.

# Initialize MessageGraph. 
graph = graphs.MessageGraph()

VAR_ORACLE = "oracle"

# Add "oracle" node to graph, calling model with input. 
graph.add_node(VAR_ORACLE, chat_models.chat_openai)
# Add edge from "oracle" to END. Execution will end after current node.
graph.add_edge(VAR_ORACLE, graphs.END)

# Set "oracle" as entrypoint to the graph. 
graph.set_entry_point(VAR_ORACLE)

# Compile graph to prevent further modifications.
runnable = graph.compile()

In [ ]:
# When executing the graph:
# LangGraph adds input message to internal state, passes state to entrypoint node "oracle".
# "Oracle" node executes, invokes chat model.
# Chat model returns AIMessage, added to state.
# Execution progresses to special END value, outputs final state.
# Result: list of two chat messages as output.
runnable.invoke(prompts.HumanMessage("What is 1 + 1?"))

## Interaction with LCEL
Add_node can take any function or runnable as input. The input to the runnable is the entire current state.



In [ ]:
def call_oracle(messages: list):
	return chat_models.chat_openai.invoke(messages)

graph.add_node(VAR_ORACLE, call_oracle)

# Documentation


# Prebuilt Example

Several methods have been added to simplify the use of common, prebuilt graphs and components.



## ToolExecutor

Class for calling tools, parameterized by a list of tools.

It exposes a runnable interface to call tools by passing in an AgentAction to look up the relevant tool and call it with the appropriate input.



In [ ]:
from langgraph.prebuilt import ToolExecutor

tools = [...]
tool_executor = ToolExecutor(tools)

## `chat_agent_executor.create_function_calling_executor`

Helper function for creating a graph for chat model using function calling. Pass in model and list of tools. Model must support OpenAI function calling.



In [ ]:
from langgraph.prebuilt import chat_agent_executor
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults
from langgraph.prebuilt import chat_agent_executor
from langchain_core.messages import HumanMessage

tools = [TavilySearchResults(max_results=1)]
model = ChatOpenAI()

app = chat_agent_executor.create_function_calling_executor(model, tools)

inputs = {"messages": [HumanMessage(content="what is the weather in sf")]}
for s in app.stream(inputs):
	print(list(s.values())[0]["messages"][0].content)
	print("----")


----
[{'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.78, 'lon': -122.42, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1714002642, 'localtime': '2024-04-24 16:50'}, 'current': {'last_updated_epoch': 1714002300, 'last_updated': '2024-04-24 16:45', 'temp_c': 16.1, 'temp_f': 61.0, 'is_day': 1, 'condition': {'text': 'Partly cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/day/116.png', 'code': 1003}, 'wind_mph': 10.5, 'wind_kph': 16.9, 'wind_degree': 290, 'wind_dir': 'WNW', 'pressure_mb': 1018.0, 'pressure_in': 30.05, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 67, 'cloud': 75, 'feelslike_c': 16.1, 'feelslike_f': 61.0, 'vis_km': 16.0, 'vis_miles': 9.0, 'uv': 5.0, 'gust_mph': 14.8, 'gust_kph': 23.8}}"}]
----
The current weather in San Francisco is partly cloudy with a temperature of 61.0°F (16.1°C). The wind speed is 16.9 km/h coming from the WNW direction. The h

## `chat_agent_executor.create_tool_calling_executor`

Helper function for creating a graph with a chat model using tool calling. Pass in model and list of tools. Model must support OpenAI tool calling.



In [ ]:
from langgraph.prebuilt import chat_agent_executor
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults
from langgraph.prebuilt import chat_agent_executor
from langchain_core.messages import HumanMessage

tools = [TavilySearchResults(max_results=1)]
model = ChatOpenAI()

app = chat_agent_executor.create_tool_calling_executor(model, tools)

inputs = {"messages": [HumanMessage(content="what is the weather in sf")]}
for s in app.stream(inputs):
	print(list(s.values())[0]["messages"][0].content)
	print("----")


----
[{"url": "https://www.weatherapi.com/", "content": "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.78, 'lon': -122.42, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1714002642, 'localtime': '2024-04-24 16:50'}, 'current': {'last_updated_epoch': 1714002300, 'last_updated': '2024-04-24 16:45', 'temp_c': 16.1, 'temp_f': 61.0, 'is_day': 1, 'condition': {'text': 'Partly cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/day/116.png', 'code': 1003}, 'wind_mph': 10.5, 'wind_kph': 16.9, 'wind_degree': 290, 'wind_dir': 'WNW', 'pressure_mb': 1018.0, 'pressure_in': 30.05, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 67, 'cloud': 75, 'feelslike_c': 16.1, 'feelslike_f': 61.0, 'vis_km': 16.0, 'vis_miles': 9.0, 'uv': 5.0, 'gust_mph': 14.8, 'gust_kph': 23.8}}"}]
----
The current weather in San Francisco is as follows:
- Temperature: 16.1°C (61.0°F)
- Condition: Partly cloudy
- Wind: 16.9 km/h from WNW
- Pressure: 1018.0 m

## Create Agent Executor

Helper function for creating LangChain Agents graph by passing in agent and list of tools.



## StateGraph

The main entrypoint, responsible for constructing the graph. The graph is parameterized by a state object passed to each node.


### `__init__`

When constructing the graph, pass in a schema for a state. Each node returns operations to update the state. Operations can SET specific attributes or ADD to existing attributes on the state, denoted by annotating the state object.

Specify the schema with a typed dictionary: `from typing import TypedDict`.

Annotate attributes using `from typing import Annotated`. Only supported annotation is `import operator; operator.add`. This annotation adds new result to existing value.

Example:

In [ ]:
class AgentState(typing.TypedDict):
	input: str
	# Outcome of call to agent. Requires `None` as valid type, as initial state
	agent_outcome: typing.Union[agents.AgentAction, agents.AgentFinish, None]
	# List of actions and corresponding observations
	# Annotate with `operator.add` to indicate operations should be added to 
	# existing values, not overwritten.
	intermediate_steps: typing.Annotated[list[tuple[agents.AgentAction, str]],
                                      operator.add]

# Initialize the StateGraph with this state
graph = graphs.StateGraph(AgentState)

# Create nodes and edges
...

# Compile the graph
app = graph.compile()

# Inputs: dictionary <- State: TypedDict
inputs = {
	"input": "hi"
	# `agent_outcome` set by the graph at some point. Defaults to None
	# `intermediate_steps` built up over time by the graph. Defaults to empty list. 
	# `intermediate_steps` annotated with `operator.add`
}

### `.add_node`

Adds a node to the graph: 
- key (a unique string representing the node's name)
- action (a function or a runnable)



### `.add_edge`

Creates an edge between nodes, passing output from the first to the next.

- start_key: Name of start node in graph.
- end_key: Name of end node in graph.

### `.add_conditional_edges`

Adds conditional edges where only one downstream edge is taken based on the start node's results. 

- `start_key`: Name of start node in graph.
- `condition`: Function to determine next edge based on start node output.
- `conditional_edge_mapping`: Maps strings (`condition`'s output) to strings (downstream nodes).


### `.set_entry_point`

The starting point of the graph, called first.

- key: Name of the node to call first.



### `.set_conditional_entry_point`

Adds a conditional entry point where the graph calls the `condition` Callable to decide the first node to enter.

- `condition`: Function to call to decide next step based on input to graph. Return string for conditional_edge_mapping.
- `conditional_edge_mapping`: Mapping of string (`condition` outputs) to string (downstream nodes to call).

### `.set_finish_point`

Final result node in the graph.

- key: Name of Node returns final output when called.

Note: Only call this if you have not created an edge leading to END.

## Graph

Similar to StateGraph but does not update a state object over time. It relies on passing the full state from each step, where the output of one node becomes the input of the next.


## END

Special node marks the end of the graph, ensuring that anything reaching it will be the final output.Possible uses:

- As `end_key` in `add_edge`
- value in `conditional_edge_mapping` for `add_conditional_edges`

# Conditional edges

Use conditional edges to route execution to a node based on the current state using a function.

Math can be difficult for LLMs, so allow them to call a "multiply" node using tool calling.

Recreate graph with "multiply" to calculate result of recent message if tool call, bind calculator to OpenAI model as tool for model to use as needed to respond to the current state.


In [ ]:
NODE_ORACLE = "oracle"
NODE_MULTIPLY = "multiply"

@agent_tools.tool
def multiply(n1: int, n2: int):
	"""
	Multiplies two numbers together.
	"""
	return n1 * n2

model = chat_models.chat_openai
tools = [
	agent_tools.convert_to_openai_tool(multiply)
]
model_with_tools = model.bind(tools=tools)

graph = graphs.MessageGraph()

def invoke_model(
	state: typing.List[prompts.BaseMessage]
):
	return model_with_tools.invoke(state)

def invoke_tool(
	state: typing.List[prompts.BaseMessage],
):
  tool_calls = state[-1].additional_kwargs.get("tool_calls", [])
  
  multiply_call = None
  
  for tool_call in tool_calls:
    if tool_call["function"]["name"] == "multiply":
      multiply_call = tool_call
  
  if multiply_call is None:
    raise Exception("No adder input found.")
  
  res = multiply.invoke(
		json.loads(multiply_call["function"]["arguments"])
	)
  
  return prompts.ToolMessage(
		tool_call_id=multiply_call["id"], content=res,
	)

def router(
	state: typing.List[prompts.BaseMessage],
) -> str:
  tool_calls = state[-1].additional_kwargs.get("tool_calls", [])
  
	# If model output has tool call, move to "multiply" node. Otherwise, end.
  if len(tool_calls):
    return NODE_MULTIPLY
  else:
    return ACTION_END

graph.add_node(NODE_ORACLE, invoke_model)
graph.add_node(NODE_MULTIPLY, invoke_tool)
graph.add_edge(NODE_MULTIPLY, graphs.END)

graph.set_entry_point(NODE_ORACLE)

# The desired outcome: If the "oracle" node returns a message expecting a tool 
# call, proceed to the "multiply" node. Otherwise, terminate execution. 
graph.add_conditional_edges(
	start_key=NODE_ORACLE, condition=router,
	conditional_edge_mapping={
		NODE_MULTIPLY: NODE_MULTIPLY,
		ACTION_END: graphs.END,
	}
)

runnable = graph.compile()

In [ ]:
runnable.invoke(prompts.HumanMessage("What is your name?"))

In [ ]:
runnable.invoke(prompts.HumanMessage("Waht is 9 * 3?"))

# Cycles

AgentExecutor uses chat models and function calling, representing all its state as a list of messages.

## Set up the tools

Define the tools to use. Wrap tools in LangGraph ToolExecutor to call and return output from ToolInvocation objects with tool and tool_input attributes.

## Set up the model

Any model supporting function calling can be selected.

Ensure the model is aware of the available tools by converting LangChain tools to OpenAI function format and binding them to the model class.

## Define Agent state

StateGraph uses a state object passed to each node, which returns operations to update the state by either setting specific attributes or adding to existing ones.

The state is a list of messages (internal state). Each node will add messages (returned values of a node) to the list using a TypedDict with one key (messages) annotated to always add with operator.add

## Define the nodes, edges

- Nodes: agent (action maker) and function (action executor for agent).

- Edges: Based on the output of a node, one of several paths may be taken. The path is determined when the node is run (the LLM decides).

	- Conditional Edge: Call agent -> 
		- ? Agent should take action -> Call function to invoke tools
		- ? Agent finished -> Finish
	- Normal Edge: Invoke tools -> Go back for next Agent decision

Example: Define nodes and a function to determine the conditional edge to take.

## Define Graph

Put everything together.

## Use it!

This exposes the same interface as other LangChain runnables. This runnable accepts a list of messages.

This process takes time as it makes background calls. Use streaming to see real-time intermediate results.



In [ ]:
tools = [
	agent_tools.TavilySearchResults(max_results=1),
]
tool_executor = graphs.ToolExecutor(tools)

#*------------------------------------------------------------------------------

model = chat_models.chat_openai

functions = [
	agent_tools.convert_to_openai_function(t) for t in tools
]
model = model.bind_functions(functions)

#*------------------------------------------------------------------------------

class AgentState(typing.TypedDict):
  messages: typing.Annotated[typing.Sequence[prompts.BaseMessage], 
                             operator.add]

#*------------------------------------------------------------------------------

def should_continue(
	state: typing.List[prompts.BaseMessage],
) -> str:
	"""
	Determines whether to continue or not
	"""
	last_message = state["messages"][-1]

	if "function_call" not in last_message.additional_kwargs:
		return ACTION_END
	else:
		return ACTION_CONTINUE

def call_model(
	state: typing.List[prompts.BaseMessage],
):
	"""
	Calls the model
	"""
	messages = state["messages"]
	response = model.invoke(messages)
 
	# return a list, because this will get added to the existing list
	return {
		"messages": [response]
	}

def call_tool(
	state: typing.List[prompts.BaseMessage],
):
	"""
	Executes tools
	"""
	messages = state["messages"]
	last_message = messages[-1]
 
	action = graphs.ToolInvocation(
		tool=last_message.additional_kwargs["function_call"]["name"],
		tool_input=json.loads(last_message.additional_kwargs["function_call"]["arguments"]),
	)
 
	response = tool_executor.invoke(action)
	function_message = prompts.FunctionMessage(content=str(response), name=action.tool)
 
	# return a list, because this will get added to the existing list
	return {
		"messages": [function_message]
	}
 
#*------------------------------------------------------------------------------

workflow = graphs.StateGraph(AgentState)

# Define two nodes for cycling.
NODE_AGENT = "agent"
NODE_ACTION = "action"

workflow.add_node(NODE_AGENT, call_model)
workflow.add_node(NODE_ACTION, call_tool)

# Set entrypoint as `agent`. The first node called.
workflow.set_entry_point(NODE_AGENT)

workflow.add_conditional_edges(
	# Define start node as `agent`, edges taken after `agent` node is called.
	NODE_AGENT,
	# Function to determine which node is called next.
	should_continue,
	# Pass in a mapping where keys are strings and values are nodes. 
	# END is a special node signifying the end of the graph. 
	# Call `should_continue` and match the output against the keys in the mapping 
	# to determine the next node.
	{
		# If `tools`, then call the tool node.
		ACTION_CONTINUE: NODE_ACTION,
		ACTION_END: graphs.END,
	}
)

# Normal edge added from `tools` to `agent`, `agent` called after `tools`.
workflow.add_edge(NODE_ACTION, NODE_AGENT)

# Compile it into a LangChain Runnable
app = workflow.compile()

In [ ]:
inputs = {
	"messages": [
		prompts.HumanMessage(content="What is the weather in sf?")
	]
}

app.invoke(inputs)

## Streaming

LangGraph supports various streaming types.



### Streaming Node Output

Stream output produced by each node.

In [ ]:
inputs = {
	"messages": [
		prompts.HumanMessage(content="What is the weather in sf?")
	]
}

for output in app.stream(inputs):
	# stream() yields dictionaries with output keyed by node name
	for key, value in output.items():
		print(f"Node: `{key}`")
		print("-"*10)
		print(value)
	print(f"{'-'*10}\n")
 

### Streaming LLM Tokens

Access LLM tokens as they are produced from each node. Only "agent" node produces LLM tokens. Use LLM supporting streaming and set it during construction (e.g. ChatOpenAI(model="gpt-3.5-turbo-1106", streaming=True)) for proper functionality.



In [ ]:
inputs = {
	"messages": [
		prompts.HumanMessage(content="What is the weather in sf?")
	]
}

async for output in app.astream_log(inputs, include_types=["llm"]):
  # astream_log() yields the requested logs (here LLMs) in JSONPatch format
	for op in output.ops:
		if op["path"] == "/streamed_output/-":
			# this is the output from .stream()
			...
		elif op["path"].startswith("/logs/") and op["path"].endswith(
			"/streamed_output/-"
		):
			# because we chose to only include LLMs, these are LLM tokens
			print(op["value"])

# How-to Guides

## [Async](https://github.com/langchain-ai/langgraph/blob/main/examples/async.ipynb)

If running LangGraph in async workflows, create nodes to be async by default. 

Example: Build a chat executor with native async implementations to take advantage of Chat Models with async clients, eliminating the need for calling the model in a separate thread.

### Tools setup

Define tools to use. 

Wrap tools in ToolExecutor: a class that takes ToolInvocation and calls the tool, returning the output. ToolInvocation has tool and tool_input attributes.

In [21]:
tools = [
	agent_tools.TavilySearchResults(max_results=1),
]
tool_executor = graphs.ToolExecutor(tools)

### Model setup

Load the chat model that meets two criteria:
- Represent all agent states in messages for it to work effectively.
- Compatible with OpenAI function calls, must be an OpenAI model or have a similar interface.

Ensure the model is aware of the available tools by converting LangChain tools to OpenAI function format and binding them to the model class.

In [22]:
model = chat_models.chat_openai
functions = [
	agent_tools.convert_to_openai_function(t) for t in tools
]
model = model.bind_functions(functions)

### Agent state definition

The main type of graph in langgraph is the StatefulGraph. This graph is parameterized by a state object that is passed to each node. Nodes return operations to update the state, either SET specific attributes or ADD to existing attributes based on the annotation of the state object.

Create a state to track a list of messages using a TypedDict with one key (messages) to ensure messages are always added.



In [23]:
class AgentState(typing.TypedDict):
	messages: typing.Annotated[typing.Sequence[prompts.BaseMessage], operator.add]

### Nodes definition

Node: a function or a runnable. Two main nodes:
- The agent: decides actions.
- Function to invoke tools: executes actions for the agent.

Edges: Based on the output of a node, one of several paths may be taken. 
- Conditional Edge: after agent call, either the function invokes tools if action instructed, or finish if agent indicates completion
- Normal Edge: after tools invoked, always return to agent for next steps

Define each node as an async function and a function to determine the conditional edge to take.

In [25]:
def should_continue(
	state: typing.List[prompts.BaseMessage],
) -> str:
	"""
	Determines whether to continue or not
 	"""
	messages = state["messages"]
	last_message = messages[-1]
	
	if ACTION_FUNCTION_CALL not in last_message.additional_kwargs:
		return ACTION_END
	else:
		return ACTION_CONTINUE


async def call_model(
	state: typing.List[prompts.BaseMessage],
):
	"""
 	Calls the model
  """
	messages = state["messages"]
	response = await model.ainvoke(messages)
	# Return a list to add to the existing list.
	return {
		"messages": [
			response,
		],
	}
 
async def call_tool(
	state: typing.List[prompts.BaseMessage],
):
	messages = state["messages"]
	# Last message involves a function call based on the continue condition.
	last_message = messages[-1]
	# Construct ToolInvocation from function_call
	action = graphs.ToolInvocation(
		tool=last_message.additional_kwargs["function_call"]["name"],
		tool_input=json.loads(
			last_message.additional_kwargs["function_call"]["arguments"]
		),
	)
	response = await tool_executor.ainvoke(action)
	function_message = prompts.FunctionMessage(content=str(response), name=action.tool)
	# Return a list to add to the existing list.
	return {
		"messages": [
			function_message,
		]
	}

### Graph definition

Define the graph by putting everything together.



In [ ]:
workflow = graphs

### Use it! -> Use it!

We can now use it, exposing the same interface as all other LangChain runnables.



### Streaming

LangGraph supports various streaming types.



#### Streaming Node Output

LangGraph makes it easy to stream output produced by each node.



#### Streamlining LLM Tokens

Access LLM tokens produced by each node. Only "agent" node produces LLM tokens. Use LLM supporting streaming and set it when constructing LLM (e.g. ChatOpenAI(model="gpt-3.5-turbo-1106", streaming=True)) for proper functionality.



In [ ]:
# TODO

## [Streaming Tokens](https://github.com/langchain-ai/langgraph/blob/main/examples/streaming-tokens.ipynb)

Sometimes language models take time to respond. Stream tokens to end users for faster responses.


In [ ]:
# TODO

## [Persistence](https://github.com/langchain-ai/langgraph/blob/main/examples/persistence.ipynb)

Save and resume graph state. 

In [ ]:
# TODO

## [Human-in-the-loop](https://github.com/langchain-ai/langgraph/blob/main/examples/human-in-the-loop.ipynb) workflows

Allow for human review the current state before moving to a specific node. 

In [ ]:
# TODO

## [Visualizing the graph](https://github.com/langchain-ai/langgraph/blob/main/examples/visualization.ipynb)

Print out and visualize the graph, generating ascii art and pngs.

In [ ]:
# TODO

## ["Time Travel"](https://github.com/langchain-ai/langgraph/blob/main/examples/time-travel.ipynb)

Modify the state at any point in the graph execution and rerun from there. This feature is beneficial for debugging workflows and correcting the state in end user-facing workflows. 

In [ ]:
# TODO

# Examples



## ChatAgentExecutor: function calling

Takes a list of messages and outputs a list of messages, with all agent state represented as a list. Ideal for chat models supporting function calling.



In [ ]:
# TODO

### [Getting Started Notebook](https://github.com/langchain-ai/langgraph/blob/main/examples/chat_agent_executor_with_function_calling/base.ipynb)

Creating executor from scratch


In [ ]:
# TODO

### [High Level Entrypoint](https://github.com/langchain-ai/langgraph/blob/main/examples/chat_agent_executor_with_function_calling/high-level.ipynb)

Using high level entrypoint for chat agent executor.


In [ ]:
# TODO


## Modifications

How to modify the base chat agent executor. 



### [Human-in-the-loop](https://github.com/langchain-ai/langgraph/blob/main/examples/chat_agent_executor_with_function_calling/human-in-the-loop.ipynb)

Adding a human-in-the-loop component


In [ ]:
# TODO

### [Force calling a tool first](https://github.com/langchain-ai/langgraph/blob/main/examples/chat_agent_executor_with_function_calling/force-calling-a-tool-first.ipynb)

Always calling a specific tool first


In [ ]:
# TODO

### [Respond in a specific format](https://github.com/langchain-ai/langgraph/blob/main/examples/chat_agent_executor_with_function_calling/respond-in-format.ipynb)

Forcing the agent to respond in a specific format


In [ ]:
# TODO

### [Dynamically returning tool output directly](https://github.com/langchain-ai/langgraph/blob/main/examples/chat_agent_executor_with_function_calling/dynamically-returning-directly.ipynb)

Allowing the agent to choose whether to return the tool output directly


In [ ]:
# TODO

### [Managing agent steps](https://github.com/langchain-ai/langgraph/blob/main/examples/chat_agent_executor_with_function_calling/managing-agent-steps.ipynb)

Explicitly managing intermediate steps taken by the agent


In [ ]:
# TODO

## Planning Agent

Agent architectures in the "plan-and-execute" style. An LLM planner decomposes a user request into a program, an executor executes the program, and an LLM synthesizes a response (and/or dynamically replans) based on the program outputs.



### [Plan-and-execute](https://github.com/langchain-ai/langgraph/blob/main/examples/plan-and-execute/plan-and-execute.ipynb)

Agent with a planner generates a multi-step task list, an executor invokes the tools in the plan, and a replanner updates the plan.



In [ ]:
# TODO

### [Reasoning without Observation](https://github.com/langchain-ai/langgraph/blob/main/examples/rewoo/rewoo.ipynb)

planner generates task list with saved observations as variables to reduce re-planning
 


In [ ]:
# TODO

### [LLMCompiler](https://github.com/langchain-ai/langgraph/blob/main/examples/llm-compiler/LLMCompiler.ipynb)

planner generates DAG of tasks with variable responses. Tasks are streamed and executed eagerly to minimize tool execution runtime



In [ ]:
# TODO

## Reflection / Self-Critique

When focusing on output quality, use self-critique and external validation to improve system outputs. 



### [Basic Reflection](https://github.com/langchain-ai/langgraph/tree/main/examples/reflection/reflection.ipynb)

Add a "reflect" step in graph to prompt system to revise its outputs.



In [ ]:
# TODO

### [Reflexion](https://github.com/langchain-ai/langgraph/tree/main/examples/reflexion/reflexion.ipynb)

Critique agent responses to guide next steps.



In [ ]:
# TODO

### [Language Agent Tree Search](https://github.com/langchain-ai/langgraph/tree/main/examples/lats/lats.ipynb)

Execute multiple agents in parallel with reflection and rewards for Monte Carlo Tree Search.



## Multi-agent



### [Multi-agent collaboration](https://github.com/langchain-ai/langgraph/blob/main/examples/multi_agent/multi-agent-collaboration.ipynb)

Creating two agents to work together to solve a problem



In [ ]:
# TODO

### [Multi-agent with supervisor](https://github.com/langchain-ai/langgraph/blob/main/examples/multi_agent/agent_supervisor.ipynb)

Orchestrating individual agents using "supervisor" LLM to distribute work



In [ ]:
# TODO

### [Hierarchical agent teams](https://github.com/langchain-ai/langgraph/blob/main/examples/multi_agent/hierarchical_agent_teams.ipynb)

Orchestrating "teams" of agents as nested graphs for collaboration to solve a problem


In [ ]:
# TODO

## Web Research



### [STORM](https://github.com/langchain-ai/langgraph/tree/main/examples/storm/storm.ipynb)

Writing system that generates Wikipedia-style articles on any topic, applying outline generation + multi-perspective question-answering for added breadth and reliability. Based on STORM by Shao, et. al.



## Chatbot Evaluation Simulation

Simulations can help evaluate chat bots. 



### [Chat bot evaluation as multi-agent simulation](https://github.com/langchain-ai/langgraph/blob/main/examples/chatbot-simulation-evaluation/agent-simulation-evaluation.ipynb)

simulate dialogue between a "virtual user" and your chat bot



In [ ]:
# TODO

### [Evaluating over a dataset](https://github.com/langchain-ai/langgraph/tree/main/examples/chatbot-simulation-evaluation/langsmith-agent-simulation-evaluation.ipynb)

benchmark your chatbot over a LangSmith dataset, tasks a simulated customer to red-team your chat bot.



In [ ]:
# TODO

## Multimodal



### [WebVoyager](https://github.com/langchain-ai/langgraph/blob/main/examples/web-navigation/web_voyager.ipynb)

Vision-enabled web browsing agent using Set-of-marks prompting for navigation and task execution.



In [ ]:
# TODO

## Chain-of-Table

Chain of Table framework answers questions on tabular data

